In [1]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from word2number import w2n
import pandas as pd
import re

In [2]:
df = pd.read_excel(r"C:\Users\ZA40142720\Downloads\validation_Spacy_trf_19072023_1147.xlsx", engine = 'openpyxl')
print(df.columns)

Index(['Statement', 'DATE_RANGE', 'ACCOUNT_NO', 'MOBILE_NO', 'DATE',
       'DELIVERABLE_TYPE', 'AMOUNT', 'CREDIT_CARD_NO', 'MODE_OF_PAYMENT',
       'CLEANED_AMOUNT', 'CLEANED_DATE'],
      dtype='object')


In [3]:
daterange = df.DATE_RANGE.dropna().tolist()

In [9]:
df = pd.read_excel(r"C:\Users\ZA40142720\Downloads\validation_Spacy_trf_19072023_1147.xlsx", engine = 'openpyxl')
daterange = df.DATE_RANGE.dropna().tolist()

pattern = r"\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b"
month_dict = {'jan': 1,'feb': 2,'mar': 3,'apr': 4,'may': 5,'jun': 6,'jul': 7,'aug': 8,'sep': 9,'oct': 10,'nov': 11,'dec': 12}

for dr_index,dr in enumerate(daterange):
    today = datetime.now().date()
    if "last" in dr:
        num_list = re.findall('[0-9]+',dr)
        if num_list == []:
            if any(word in dr for word in ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']):
                nums = w2n.word_to_num(dr)
            else:
                nums = 1
        else:
            nums = int(num_list[0])
        matches = re.findall(pattern,dr)
        if matches == []:
            word_month = False
        else:
            word_month = True
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
   
        if "year" in dr or "yr" in dr or "financial year" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
            start_date = today - relativedelta(years=nums)
            start_date = datetime(start_date.year,4,1).date()
            end_date = start_date + relativedelta(years=nums, days=-1)
            end_date = datetime(end_date.year,3,31).date()

        elif "month" in dr or "mnth" in dr:
            if word_month == False:
                start_date = today - relativedelta(months=nums)
                end_date = start_date + relativedelta(months=nums, days=-1)
            elif word_month == True:
                start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
                end_date = start_date + relativedelta(months=1, days=-1)

        elif "day" in dr:
            start_date = today - relativedelta(days=nums)
            end_date = start_date + relativedelta(days=nums)

    elif "last" not in dr and "from" not in dr and "to" not in dr and "-" not in dr:

        num_list = re.findall('[0-9]+',dr)
        if num_list == []:
            if any(word in dr for word in ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']):
                nums = w2n.word_to_num(dr)
            else:
                nums = 1
            non_num_flag = True
        else:
            nums = int(num_list[0])
            non_num_flag = False

        matches = re.findall(pattern,dr)
        if matches == []:
            word_month = False
        else:
            word_month = True
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
        if "month" in dr or "mnth" in dr:
            if word_month == False:
                if "this" in dr or "current" in dr:
                    start_date = today - relativedelta(days=today.day-1)
                    end_date = start_date + relativedelta(months=1, days=-1)
                else:
                    start_date = today - relativedelta(months=nums)
                    end_date = start_date + relativedelta(months=nums, days=-1)
            elif word_month == True:
                start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
                end_date = start_date + relativedelta(months=1, days=-1)

        elif "annual" in dr or "financial" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
            if non_num_flag == False:
                num_index_flag, word_index_flag = False, False
                for i,w in enumerate(dr.split()):
                    if str(nums) in w:
                        num_index = i
                        num_index_flag = True
                    if "annual" in dr or "financial" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
                        word_index = i
                        word_index_flag = True
                if num_index_flag & word_index_flag:
                    if num_index >= word_index:
                        nums = 1          
            start_date = today - relativedelta(years=nums)
            start_date = datetime(start_date.year,4,1).date()
            end_date = start_date + relativedelta(years=nums, days=-1)
            end_date = datetime(end_date.year,3,31).date()

        elif len(re.findall(pattern,dr)) > 0:
            matches = re.findall(pattern,dr)
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
            start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
            end_date = start_date + relativedelta(months=1, days=-1)
            print("---->>>",dr_index, dr, start_date, end_date)


---->>> 15 05th jan 2023-01-01 2023-01-31
---->>> 48 for february . 2023-02-01 2023-02-28
---->>> 52 in november 2023-11-01 2023-11-30
---->>> 54 1st. april  till . march . 2023-04-01 2023-04-30
---->>> 84 for may and june. 2023-05-01 2023-05-31
---->>> 113 estatement for may .

 2023-05-01 2023-05-31
---->>> 128 for may on 2023-05-01 2023-05-31
---->>> 130 28th of jun. 2023-06-01 2023-06-30
---->>> 132 may`23 2023-05-01 2023-05-31
---->>> 144 april 03, 2023-04-01 2023-04-30
---->>> 165 end of june. 2023-06-01 2023-06-30
---->>> 197 starting jan  till  august . 2023-01-01 2023-01-31
---->>> 212 starting jan  till  august . 2023-01-01 2023-01-31
---->>> 227 31st march 2023-03-01 2023-03-31
---->>> 229 3 jun, 2023-06-01 2023-06-30
---->>> 238 3 june 2023-06-01 2023-06-30
---->>> 242 date of 5th june 2023-06-01 2023-06-30
---->>> 244 in oct 2023-10-01 2023-10-31
---->>> 247 year ended 31 march 2023-03-01 2023-03-31
---->>> 267 till 31st may 2023-05-01 2023-05-31
---->>> 272 june 8 2023-06

In [54]:
dr = "month of may23"
re.findall(pattern,dr)

[]

In [34]:
# import re
# a = "arunav 123 sahay123"
# re.findall('[0-9]+',a)

today = datetime.now().date()
print(today.year)
{m:index+1 for index,m in enumerate(month_dict)}

2023


{'jan': 1,
 'feb': 2,
 'mar': 3,
 'apr': 4,
 'may': 5,
 'jun': 6,
 'jul': 7,
 'aug': 8,
 'sep': 9,
 'oct': 10,
 'nov': 11,
 'dec': 12}

In [ ]:
import pandas as pd
import re
from word2number import w2n
from datetime import datetime
from dateutil.relativedelta import relativedelta

def word_to_number(word):
    # ... (same as your word_to_number function)
    # ... (omitting for brevity)

def join_text_by_index(str_date):
    # ... (same as your join_text_by_index function)
    # ... (omitting for brevity)

def process_date_range(df):
    pattern = r"\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b"
    month_dict = {'jan': 1,'feb': 2,'mar': 3,'apr': 4,'may': 5,'jun': 6,'jul': 7,'aug': 8,'sep': 9,'oct': 10,'nov': 11,'dec': 12}

    daterange = df.DATE_RANGE.dropna().tolist()
    today = datetime.now().date()

    for dr_index, dr in enumerate(daterange):
        # ... (same as your existing code)
        # ... (omitting for brevity)

        elif list(join_text_by_index(dr)):
            print(join_text_by_index(dr))

# Load the Excel file and call the function
df = pd.read_excel(r"C:\Users\ZA40142720\Downloads\validation_Spacy_trf_19072023_1147.xlsx", engine='openpyxl')
process_date_range(df)


In [2]:
import re
import pandas as pd 
from word2number import w2n
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from datetime import datetime, timedelta


def word_to_number(word):
    ordinals = {
        'first': 1, '1st': 1, 'second': 2, '2nd': 2, '3rd': 3, 'third': 3, 'fourth': 4, '4th': 4,
        'fifth': 5, '5th': 5, 'sixth': 6, '6th': 6, 'seventh': 7, '7th': 7, 'eighth': 8, '8th': 8,
        'ninth': 9, '9th': 9, 'tenth': 10, '10th': 10
    }
    number = ordinals.get(word.lower())
    if number is not None:
        # print(number)
        return str(number)
    try:
        number = w2n.word_to_num(word)
        # print(number)
        return str(number)
    except ValueError:
        return None
        

def join_text_by_index(str_date):
    split_val = re.split('(\d+)', str_date)
    # print(split_val)
    index_list = []
    parse_result_list = []
    
    matching_list = ['tilltoday', 'till date', 'till today', 'till now', 'untill now', 'untill today']
    matching_pattern = '|'.join(re.escape(match) for match in matching_list)
    matching_regex = re.compile(rf'(?i)\b(?:{matching_pattern})\b')
    
    parsed_date_list = []
    month_pattern = r'(?i)jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?'
    current_date = 'to'+(datetime.now().strftime("%d-%m-%Y"))
    for i, j in enumerate(split_val):
        j = j.strip()
        exception_j_list = j.lower().split()
        matches = matching_regex.findall(j)
    
        if j.isdigit():
            parse_result_list.append(j)
            index_list.append(i) 
        
        elif re.match(month_pattern, j):
            parse_result_list.append(j)
            index_list.append(i)
        
        elif len(exception_j_list) > 1:
            
            for q in exception_j_list:
                match = re.match(month_pattern, q)
                pattern_to = r'\bto\b'
                match_to = re.findall(pattern_to, q, re.IGNORECASE)
                
                if match:
                    split_val[i] = split_val[i].replace(split_val[i], match.group(), 1)
                    parse_result_list.append(q)
                    index_list.append(i)
        
                elif matches:
                    split_val[i] = split_val[i].replace(split_val[i],current_date)            
                    parse_result_list.append(q)
                    index_list.append(i)
                elif match_to:
                    split_val[i] = split_val[i].replace(split_val[i],q)            
                    parse_result_list.append(q)
                    index_list.append(i)
          
                elif word_to_number(q):
                    split_val[i] = split_val[i].replace(split_val[i], word_to_number(q))
                    parse_result_list.append(q)
                    index_list.append(i)
                       
        elif j.strip().lower() in matching_list:
            split_val[i] = split_val[i].replace(split_val[i], match.group(), 1)
            parse_result_list.append(j)
            index_list.append(i)
    if len(index_list)>1 and len(parse_result_list)>1:
        start_index = index_list[0]
        end_index = index_list[-1] + 1
        
        joined_text = ''.join(split_val[start_index:end_index])
        joined_text_list = joined_text.split("to")
        # print("======================",joined_text_list,"========================")
        try:
            for i in joined_text_list:
                parsed_date = parser.parse(i.strip(), fuzzy=True, dayfirst = True, default = None, ignoretz = True)
                # print("parsed_date",parsed_date)
                parsed_date_list.append(parsed_date.strftime("%d-%m-%Y"))
                # print("parsed_date_list",parsed_date_list)
            return [' '.join(parsed_date_list)]
        except ValueError:
            return joined_text
    else:
        return str_date




def process_date_range(dr):
    pattern = r"\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\b"
    month_dict = {'jan': 1,'feb': 2,'mar': 3,'apr': 4,'may': 5,'jun': 6,'jul': 7,'aug': 8,'sep': 9,'oct': 10,'nov': 11,'dec': 12}
    today = datetime.now().date()
    if pd.isna(dr):
        return None
    if "last" in dr:
        num_list = re.findall('[0-9]+',dr)
        if num_list == []:
            if any(word in dr for word in ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']):
                nums = w2n.word_to_num(dr)
            else:
                nums = 1
        else:
            nums = int(num_list[0])
        matches = re.findall(pattern,dr)
        if matches == []:
            word_month = False
        else:
            word_month = True
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
   
        if "year" in dr or "yr" in dr or "financial year" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
            start_date = today - relativedelta(years=nums)
            start_date = datetime(start_date.year,4,1).date()
            end_date = start_date + relativedelta(years=nums, days=-1)
            end_date = datetime(end_date.year,3,31).date()
            return start_date,end_date

        elif "month" in dr or "mnth" in dr:
            if word_month == False:
                start_date = today - relativedelta(months=nums)
                end_date = start_date + relativedelta(months=nums, days=-1)
                return start_date,end_date
            elif word_month == True:
                start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
                end_date = start_date + relativedelta(months=1, days=-1)
                return start_date,end_date

        elif "day" in dr:
            start_date = today - relativedelta(days=nums)
            end_date = start_date + relativedelta(days=nums)
            return start_date,end_date

    elif "last" not in dr and "from" not in dr and "to" not in dr and "-" not in dr:

        num_list = re.findall('[0-9]+',dr)
        if num_list == []:
            if any(word in dr for word in ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']):
                nums = w2n.word_to_num(dr)
            else:
                nums = 1
            non_num_flag = True
        else:
            nums = int(num_list[0])
            non_num_flag = False

        matches = re.findall(pattern,dr)
        if matches == []:
            word_month = False
        else:
            word_month = True
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
        if "month" in dr or "mnth" in dr:
            if word_month == False:
                if "this" in dr or "current" in dr:
                    start_date = today - relativedelta(days=today.day-1)
                    end_date = start_date + relativedelta(months=1, days=-1)
                    return start_date,end_date
                else:
                    start_date = today - relativedelta(months=nums)
                    end_date = start_date + relativedelta(months=nums, days=-1)
                    return start_date,end_date
            elif word_month == True:
                start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
                end_date = start_date + relativedelta(months=1, days=-1)
                return start_date,end_date
        elif "annual" in dr or "financial" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
            if non_num_flag == False:
                num_index_flag, word_index_flag = False, False
                for i,w in enumerate(dr.split()):
                    if str(nums) in w:
                        num_index = i
                        num_index_flag = True
                    if "annual" in dr or "financial" in dr or "fy" in dr or "f.y" in dr or "f y" in dr:
                        word_index = i
                        word_index_flag = True
                if num_index_flag & word_index_flag:
                    if num_index >= word_index:
                        nums = 1          
            start_date = today - relativedelta(years=nums)
            start_date = datetime(start_date.year,4,1).date()
            end_date = start_date + relativedelta(years=nums, days=-1)
            end_date = datetime(end_date.year,3,31).date()
            return start_date,end_date
        elif len(re.findall(pattern,dr)) > 0:
            matches = re.findall(pattern,dr)
            for short_months in month_dict.keys():
                if short_months in matches[0]:
                    month_number = month_dict[short_months]
            start_date = today - relativedelta(months=today.month-month_number, days=today.day-1)
            end_date = start_date + relativedelta(months=1, days=-1)
            # print("---->>>",dr_index, dr, start_date, end_date)
            return start_date,end_date
    elif list(join_text_by_index(dr)):
        # print(join_text_by_index(dr))
        return join_text_by_index(dr)

df = pd.read_excel(r"C:\Users\ZA40142720\Downloads\validation_Spacy_trf_19072023_1147.xlsx", engine = 'openpyxl')
df.head()

# df['DATE_RANGE'] = df['DATE_RANGE'].apply(lambda dr: process_date_range(dr))

df['DATE_RANGE'].dropna(inplace=True)
df['CLEANED_DATE_RANGE'] = df['DATE_RANGE'].apply(lambda dr: process_date_range(dr))
df['CLEANED_DATE_RANGE']

NameError: name 'parser' is not defined

In [13]:
# from dateutil.parser import parse
!pip install python-dateutil
# !pip install --upgrade pip


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from dateutil.parser import parse as duparse
from datetime import datetime
# parsed_date = parser.parse('8/9/20', fuzzy=True, dayfirst = True, default = None, ignoretz = True)

parsed_date = parser.parse('8/9/20')
parsed_date

NameError: name 'parser' is not defined

In [ ]:
from dateutil.parser import parse as duparse
from datetime import datetime

def parse(dtstr, *args, **kwargs):
    if len(dtstr) == 8 and dtstr.isnumeric():
        return datetime.strptime(dtstr, '%d%m%Y')
    else:
        return duparse(dtstr, *args, **kwargs)